In [5]:
import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector
from scipy.spatial.distance import cosine
from numpy.linalg import norm

In [6]:
filename = "data/all.txt"

vocabulary = tf.compat.as_str(open(filename, encoding='utf8').read()).split()
vocab = list(set(vocabulary))


# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = len(vocab)
print('Data size', len(vocabulary), ' Vocab Size', vocabulary_size)


def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary



Data size 10193286  Vocab Size 610151


In [7]:
data, count, dictionary, reverse_dictionary = build_dataset(
    vocabulary, vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0



Most common words (+UNK) [['UNK', 1], ('*', 604847), ('ነው', 168852), ('#', 94645), ('ላይ', 91553)]
Sample data [696, 2783, 1446, 59726, 2388, 97389, 1284, 826, 2150, 191] ['ጋዜጠኛ', 'ተመስገን', 'ደሳለኝ', 'በጠበቃው', 'በአቶ', 'አምሐ', 'መኮንን', 'አማካይነት', 'በፌዴራል', 'ጠቅላይ']


In [8]:

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
#     batch_size = batch_size // 2
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(
        maxlen=span)  # pylint: disable=redefined-builtin
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
            
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels


batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=2)
for i in range(len(batch)):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0],
          reverse_dictionary[labels[i, 0]])

# Step 4: Build and train a skip-gram model.


1446 ደሳለኝ -> 696 ጋዜጠኛ
1446 ደሳለኝ -> 2388 በአቶ
59726 በጠበቃው -> 2388 በአቶ
59726 በጠበቃው -> 1446 ደሳለኝ
2388 በአቶ -> 1284 መኮንን
2388 በአቶ -> 1446 ደሳለኝ
97389 አምሐ -> 2388 በአቶ
97389 አምሐ -> 826 አማካይነት


In [9]:

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1  # How many words to consider left and right.
num_skips = 2  # How many times to reuse an input to generate a label.
num_sampled = 64  # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
np.random.seed(1000)
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
print(valid_examples[:10])


[88  4 62 63 48 85 73 84 39 47]


In [10]:

graph = tf.Graph()

with graph.as_default():

    # Input data.
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
        

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        with tf.name_scope('embeddings'):
            embeddings = tf.Variable(
                tf.random_normal([vocabulary_size, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)    
            print(embed.get_shape())

        # Construct the variables for the NCE loss
        with tf.name_scope('weights'):
            nce_weights = tf.Variable(
                tf.truncated_normal(
                    [vocabulary_size, embedding_size],
                    stddev=1.0 / math.sqrt(embedding_size)))
        with tf.name_scope('biases'):
            nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    # Explanation of the meaning of NCE loss:
    #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=train_labels,
                inputs=embed,
                num_sampled=num_sampled,
                num_classes=vocabulary_size))

    # Add the loss value as a scalar to summary.
    tf.summary.scalar('loss', loss)

    # Construct the SGD optimizer using a learning rate of 1.0.
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norms = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norms
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Merge all summaries.
    merged = tf.summary.merge_all()

    # Add variable initializer.
    init = tf.global_variables_initializer()

    # Create a saver.
    saver = tf.train.Saver()


(128, 128)


In [12]:
num_steps = 200001

with tf.Session(graph=graph) as session:
    # Open a writer to write summaries.
    writer = tf.summary.FileWriter('./log', session.graph)

    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips,
                                                    skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # Define metadata variable.
        run_metadata = tf.RunMetadata()

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        # Also, evaluate the merged op to get all summaries from the returned "summary" variable.
        # Feed metadata variable to session for visualizing the graph in TensorBoard.
        _, summary, loss_val = session.run(
            [optimizer, merged, loss],
            feed_dict=feed_dict,
            run_metadata=run_metadata)
        average_loss += loss_val

        # Add returned summaries to writer in each step.
        writer.add_summary(summary, step)
        # Add metadata to visualize the graph for the last run.
        if step == (num_steps - 1):
            writer.add_run_metadata(run_metadata, 'step%d' % step)

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
#                 print(log_str)
    final_embeddings = normalized_embeddings.eval()
    
#     lines = open('semantic.txt', encoding='utf8').readlines()
# #     print(list(dictionary.keys())[10])
#     s = ''
#     correct = 0
#     for line in lines:
#         words = line[:-1].split(' ')
#         target = dictionary[words[3]]
#         ix = np.array([dictionary[words[0]],
#                         dictionary[words[1]],
#                         dictionary[words[2]]]).reshape((3, 1))
#         m = session.run([hypemax], feed_dict={ixa:ix[0], ixb:ix[1], ixc:ix[2]})
#         index = m[0][0]
#         rword = reverse_dictionary[index]
#         if index == target:
#             correct += 1
        
#         s += str(words) + ' ' + rword + '\n'
#     open('emb_result.txt', mode='w', encoding='utf-8').write(s)
#     print(correct/len(lines))
        

    # Write corresponding labels for the embeddings.
    with open('./log/metadata.tsv', 'w', encoding='utf8') as f:
        for i in xrange(vocabulary_size):
            f.write(reverse_dictionary[i] + '\n')

    # Save the model for checkpoints.
    saver.save(session, os.path.join('./log', 'model.ckpt'))

    # Create a configuration for visualizing embeddings with the labels in TensorBoard.
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = embeddings.name
    embedding_conf.metadata_path = os.path.join('./log', 'metadata.tsv')
    projector.visualize_embeddings(writer, config)

writer.close()

Initialized
Average loss at step  0 :  345.43255615234375
Average loss at step  2000 :  257.212796333313
Average loss at step  4000 :  193.6795955352783
Average loss at step  6000 :  168.03714843559266
Average loss at step  8000 :  149.56211748313905
Average loss at step  10000 :  134.94462899780274
Average loss at step  12000 :  123.217674325943
Average loss at step  14000 :  115.5767941608429
Average loss at step  16000 :  107.96540307998657
Average loss at step  18000 :  101.18615557098389
Average loss at step  20000 :  95.44731834888458
Average loss at step  22000 :  88.84101161527634
Average loss at step  24000 :  85.85769393539428
Average loss at step  26000 :  82.68102250528335
Average loss at step  28000 :  78.61551406097412
Average loss at step  30000 :  74.5018870716095
Average loss at step  32000 :  70.44010382652283
Average loss at step  34000 :  68.9259870467186
Average loss at step  36000 :  64.69377996301651
Average loss at step  38000 :  62.60387661886215
Average loss a

In [13]:

def analogy(embeds, norms, a, b, c, d):
    ia, ib, ic, iid = dictionary[a], dictionary[b], dictionary[c], dictionary[d]
    xa, xb, xc, xd = embeds[ia], embeds[ib], embeds[ic], embeds[iid]
    xtemp = xb - xa + xc
    sims = 1 - np.matmul(xtemp, embeds.T)/norm(xtemp) #/ (norms)
    m = np.argmax(sims)
    return reverse_dictionary[m]

def nearest(embeds, a, b, c):
    ia, ib, ic = dictionary[a], dictionary[b], dictionary[c]
    xa, xb, xc = embeds[ia], embeds[ib], embeds[ic]
    xtemp = xb - xa + xc
    sims =  np.matmul(xtemp, embeds.T)
    argi = (-sims).argsort()[:5]
    argNearest = sims[argi]
    nearest = [reverse_dictionary[i] for i in argi]
    return nearest, argNearest

def nearto(embedding, word):
    wi = dictionary[word]
    xw = embedding[wi]
    sims = np.matmul(xw, embedding.T)
    argi = (-sims).argsort()[1:6]
    argNearest = sims[argi]
    nearest = [reverse_dictionary[i] for i in argi]
    return nearest, argNearest

In [14]:
words = "ከፍተኛ ዝቅተኛ ትልቅ አነስተኛ".split(' ')
print(nearest(final_embeddings, words[0], words[1],words[2]))
print(nearto(final_embeddings, "እላይ"))

(['ዝቅተኛ', 'ትልቅ', 'ሌላ', 'ኑሮን', 'የሚመጣ'], array([0.93904656, 0.6362028 , 0.5213963 , 0.51903737, 0.50394785],
      dtype=float32))
(['እማኖረው', 'በሚያሳጣን', 'የጠላትህን', 'ሰቅለውኝ', 'የደንገል'], array([0.6544908 , 0.65212256, 0.64374524, 0.64108545, 0.63616264],
      dtype=float32))


In [16]:

lines = open('data/semantic.txt', encoding='utf8').readlines()
s = ''
correct = 0
norms = np.linalg.norm(final_embeddings, axis=1)
for line in lines:
    words = line[:-1].split(' ')
    pred = analogy(final_embeddings, norms, words[0], words[1], words[2], words[3])
    s += str(words) + ' ' + pred + '\n'
    if words[-1] == pred:
        correct += 1
    
open('results/emb_result.txt', mode='w', encoding='utf-8').write(s)
print("Correct: {0} Accuracy: {1:.02}%".format(correct, (correct * 100 / len(lines))))

Correct: 0 Accuracy: 0.0%


In [ ]:

# pylint: disable=missing-docstring
# Function to draw visualization of distance between embeddings.
def plot_with_labels(low_dim_embs, labels, filename):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.rc('font', family='Abyssinica SIL')
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(
            label,
            xy=(x, y),
            xytext=(5, 2),
            textcoords='offset points',
            ha='right',
            va='bottom')

    plt.savefig(filename)


try:
    # pylint: disable=g-import-not-at-top
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(
        perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels, 'tsne.png')

except ImportError as ex:
    print('Please install sklearn, matplotlib, and scipy to show embeddings.')
    print(ex)
